In [1]:
import torch

# Define the labels (5x5 arrays)
x_tensor = torch.tensor([
    [
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 1, 1, 1, 0],
    ],
    [
        [1, 1, 1, 1, 1],
        [0, 0, 0, 0, 1],
        [1, 1, 1, 1, 1],
        [1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1],
    ],
    [
        [1, 1, 1, 1, 1],
        [0, 0, 0, 0, 1],
        [1, 1, 1, 1, 1],
        [0, 0, 0, 0, 1],
        [1, 1, 1, 1, 1],
    ],
    [
        [0, 0, 0, 0, 1],
        [0, 0, 1, 0, 1],
        [1, 1, 1, 1, 1],
        [0, 0, 0, 0, 1],
        [0, 0, 0, 0, 1],
    ],
    [
        [1, 1, 1, 1, 1],
        [1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1],
        [0, 0, 0, 0, 1],
        [1, 1, 1, 1, 1],
    ]
], dtype= torch.float)

# Corresponding labels (0-5)
targets = torch.tensor([
    0,
    1,
    2,
    3,
    4
])

In [2]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, x_tensor: torch.Tensor, y_tensor: torch.Tensor):
        super(CustomDataset, self).__init__()

        self.x = x_tensor
        self.y = y_tensor

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]


In [3]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [4]:
from torch.utils.data import DataLoader

train_data = CustomDataset(
    x_tensor=x_tensor,
    y_tensor=targets
)

test_data = CustomDataset(
    x_tensor, targets
)

train_dataloader = DataLoader(train_data, batch_size=5,shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=5, shuffle=True)

In [5]:
from torch import nn

class Perceptron(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_softmax_stack = nn.Sequential(
            nn.Linear(5*5, 5),
            nn.Softmax(dim=1),
        )

    def forward(self, x):
        x = self.flatten(x)
        outputs = self.linear_softmax_stack(x)
        return outputs


In [6]:
model = Perceptron().to(device)
print(model)

Perceptron(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_softmax_stack): Sequential(
    (0): Linear(in_features=25, out_features=5, bias=True)
    (1): Softmax(dim=1)
  )
)


In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

In [8]:
def train_loop():
    model.train()
    for X, y in train_dataloader:
        X = X.to(device)
        y = y.to(device)
        
        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    print(loss.item())


def test_loop():
    model.eval()
    
    for X, y in test_dataloader:
        X = X.to(device)
        y = y.to(device)

        with torch.no_grad():
            pred = model(X)
        correct = (pred.argmax(1) == y).type(torch.float).sum().item() / pred.shape[0]
        print(correct*100)

epochs = 1000
for i in range(epochs):
    print(f"Epoch {i+1} \n------------------")
    train_loop()
    test_loop()

Epoch 1 
------------------
1.5964311361312866
40.0
Epoch 2 
------------------
1.595301866531372
40.0
Epoch 3 
------------------
1.594172716140747
40.0
Epoch 4 
------------------
1.5930440425872803
40.0
Epoch 5 
------------------
1.5919158458709717
40.0
Epoch 6 
------------------
1.5907881259918213
40.0
Epoch 7 
------------------
1.58966064453125
40.0
Epoch 8 
------------------
1.5885329246520996
40.0
Epoch 9 
------------------
1.5874046087265015
40.0
Epoch 10 
------------------
1.5862751007080078
40.0
Epoch 11 
------------------
1.5851430892944336
40.0
Epoch 12 
------------------
1.5840075016021729
40.0
Epoch 13 
------------------
1.5828676223754883
40.0
Epoch 14 
------------------
1.5817219018936157
40.0
Epoch 15 
------------------
1.5805699825286865
40.0
Epoch 16 
------------------
1.5794109106063843
40.0
Epoch 17 
------------------
1.578244686126709
40.0
Epoch 18 
------------------
1.5770704746246338
40.0
Epoch 19 
------------------
1.5758883953094482
40.0
Epoch 2

In [17]:
torch.set_printoptions(sci_mode=False)
model(x_tensor.to(device))*100

tensor([[   97.98111,     0.36212,     0.78873,     0.33541,     0.53263],
        [    0.08210,    85.07806,    11.71032,     0.56541,     2.56411],
        [    0.07864,    12.29624,    75.31409,     1.05182,    11.25920],
        [    0.08567,     0.41871,     1.20866,    97.68792,     0.59905],
        [    0.09537,     3.11826,     9.74108,     0.50340,    86.54190]],
       device='cuda:0', grad_fn=<MulBackward0>)